In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

from bs4 import BeautifulSoup
import time
import grequests
import pickle

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import pickle
import os
import torch
import numpy as np
import pandas as pd

import torch
# from transformers import AutoTokenizer, AutoModel

from PIL import Image
from torch.utils.data import Dataset
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from pathlib import Path
from tqdm import tqdm
from mmpfn.models.dino_v2.models.vision_transformer import vit_base

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [3]:

directory = 'data/channel/'
filename = 'OnlineNewsPopularity.csv'
problem_type = 'regression'
label = 'log_shares'

seed = 123
output_subdir = 'processed/' # where to write files
train_name = 'train.csv'
test_name = 'test.csv'

all_train_data = pd.read_csv(directory+filename)



num_scrape = len(all_train_data)
article_titles = [''] * num_scrape
urls = list(all_train_data['url'][:num_scrape])
outer_loop = 400
j = 0
upper_index = 0
while upper_index < num_scrape:
    lower_index = j*outer_loop
    upper_index = min(lower_index + outer_loop, num_scrape)
    urls_j = list(urls[lower_index:upper_index])
    rs = (grequests.get(u, timeout=0.5) for u in urls_j)
    for inner_i, resp in enumerate(grequests.map(rs, size=100)):
        i = lower_index + inner_i
        if resp is not None:
            soup = BeautifulSoup(resp.text, 'html.parser')
            titles = soup.find_all('title')
            if titles is not None and len(titles) > 0 and titles[0] is not None and len(titles[0]) > 0:
                try:
                    article_titles[i] = titles[0].text
                except Exception as e:
                    print(f"could not parse: {urls[i]}")
            else:
                print(f"could not parse: {urls[i]}")
        else:
            print(f"could not parse: {urls[i]}")
        if i % 500 == 0:
            print(f"processed {i} URLS of {len(all_train_data)}")
    print(f"j= {j} out of num_scrape/outer_loop")
    time.sleep(0.1)
    pickle.dump( article_titles, open( os.path.join(directory, "article_titles.p"), "wb" ) )
    j += 1

processed 0 URLS of 39644
could not parse: http://mashable.com/2013/01/09/1600-penn-sprinkles-cupcakes/
could not parse: http://mashable.com/2013/01/10/shirtless-guys-water-balloons/
j= 0 out of num_scrape/outer_loop
could not parse: http://mashable.com/2013/01/14/netflix-cartoon-network-adult-swim/
could not parse: http://mashable.com/2013/01/14/samsung-85-inch-ultrahd-tv-korea/
could not parse: http://mashable.com/2013/01/14/sinofsky-iphone/
could not parse: http://mashable.com/2013/01/14/top-comments-jan13/
could not parse: http://mashable.com/2013/01/15/adam-carolla-launches-video-podcast-network-on-youtube/
could not parse: http://mashable.com/2013/01/15/china-51-million-2012/
could not parse: http://mashable.com/2013/01/15/color-scheme-tools/
could not parse: http://mashable.com/2013/01/15/curiosity-rover-first-drill-mars/
could not parse: http://mashable.com/2013/01/15/dr-oz-social-health-verticals/
could not parse: http://mashable.com/2013/01/15/sandy-hook-over-the-rainbow-newt

In [17]:
all_train_data.article_title

0        Amazon's Streaming Video Library Now a Little ...
1        AP's Twitter to Begin Displaying Sponsored Twe...
2        Apple's App Store Passes 40 Billion Downloads ...
3        This Astronaut Is Rooting for Notre Dame Tonig...
4        New U-Verse Apps Simplify Sharing Photos and V...
                               ...                        
39572    That time when Obama was photographed wearing ...
39593    'Saved By the Bell' actor Dustin Diamond arres...
39599    Two Hong Kong highway cash grabbers arrested |...
39600    Marshall Woburn: This Bluetooth speaker is bui...
39610    Russia dubs NATO major threat in new military ...
Name: article_title, Length: 6472, dtype: object

In [18]:
df = pd.read_csv(directory+filename)

In [23]:
for k, v in df.dtypes.items():
    print(k, v)

url object
 timedelta float64
 n_tokens_title float64
 n_tokens_content float64
 n_unique_tokens float64
 n_non_stop_words float64
 n_non_stop_unique_tokens float64
 num_hrefs float64
 num_self_hrefs float64
 num_imgs float64
 num_videos float64
 average_token_length float64
 num_keywords float64
 data_channel_is_lifestyle float64
 data_channel_is_entertainment float64
 data_channel_is_bus float64
 data_channel_is_socmed float64
 data_channel_is_tech float64
 data_channel_is_world float64
 kw_min_min float64
 kw_max_min float64
 kw_avg_min float64
 kw_min_max float64
 kw_max_max float64
 kw_avg_max float64
 kw_min_avg float64
 kw_max_avg float64
 kw_avg_avg float64
 self_reference_min_shares float64
 self_reference_max_shares float64
 self_reference_avg_sharess float64
 weekday_is_monday float64
 weekday_is_tuesday float64
 weekday_is_wednesday float64
 weekday_is_thursday float64
 weekday_is_friday float64
 weekday_is_saturday float64
 weekday_is_sunday float64
 is_weekend float64
 LD

In [5]:




directory = 'data/channel/'
filename = 'OnlineNewsPopularity.csv'
label = 'channel'

problem_type = 'multiclass'
eval_metric='acc'

seed = 123
output_subdir = 'processed/' # where to write files
train_name = 'train.csv'
test_name = 'test.csv'

all_train_data = pd.read_csv(directory+filename)

article_titles = pickle.load(open(os.path.join(directory, "article_titles.p"), "rb" ) )

all_train_data['article_title'] = article_titles



channels = [' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world']

channel_ohe = all_train_data[channels]

def get_channel(row):
    for c in channel_ohe.columns:
        if row[c]==1:
            return c

channel_data = channel_ohe.apply(get_channel, axis=1)
all_train_data[label] = channel_data

# check labels:
all_train_data[label].value_counts()
print("missing label values: ", np.sum(all_train_data[label].isna()))



# Keep rows
keep_ind = (all_train_data['article_title'] != '') & ~all_train_data[label].isna()
all_train_data = all_train_data[keep_ind]

# Keep columns:
bad_columns = ['url', ' timedelta', ' n_tokens_title', ' title_subjectivity', ' title_sentiment_polarity',
               ' abs_title_subjectivity', ' abs_title_sentiment_polarity',
              ' avg_positive_polarity', ' min_positive_polarity', ' max_positive_polarity', ' avg_negative_polarity', ' min_negative_polarity', ' max_negative_polarity',
               ' kw_min_min', ' kw_max_min', ' kw_avg_min',
               ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
               ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
               ' self_reference_max_shares', ' self_reference_avg_sharess',
               ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
               ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
               ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02',
               ' LDA_03', ' LDA_04', ' shares',
               ' data_channel_is_lifestyle', ' data_channel_is_entertainment',
               ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world',
               ]
all_train_data.drop(columns=bad_columns, inplace=True)


# Split
train_data, test_data = train_test_split(all_train_data,
                                        test_size=0.2,
                                        stratify=all_train_data[label],
                                        random_state=np.random.RandomState(seed))

train_data.to_csv(os.path.join(directory, train_name), index=False)
test_data.to_csv(os.path.join(directory, test_name), index=False)
print(f'#Train={len(train_data)}, #Dev={len(test_data)}')


# Test run autogluon:



# MAX_NGRAM = 300
# time_limit = 300
# feature_generator = AutoMLPipelineFeatureGenerator(vectorizer=CountVectorizer(min_df=30, ngram_range=(1, 3), max_features=MAX_NGRAM, dtype=np.uint8))

# predictor = TabularPredictor(label=label, path=directory+output_subdir, problem_type=problem_type)
# predictor.fit(train_data, time_limit=time_limit, feature_generator=feature_generator, hyperparameters={'GBM':{}})
# predictor.evaluate(test_data)

# Compute checksum:



missing label values:  6134
#Train=5177, #Dev=1295


In [6]:
train_data.shape

(5177, 18)

In [8]:
train_data.dtypes

 n_tokens_content              float64
 n_unique_tokens               float64
 n_non_stop_words              float64
 n_non_stop_unique_tokens      float64
 num_hrefs                     float64
 num_self_hrefs                float64
 num_imgs                      float64
 num_videos                    float64
 average_token_length          float64
 num_keywords                  float64
 global_subjectivity           float64
 global_sentiment_polarity     float64
 global_rate_positive_words    float64
 global_rate_negative_words    float64
 rate_positive_words           float64
 rate_negative_words           float64
article_title                   object
channel                         object
dtype: object

In [12]:
train_data.channel.unique()

array([' data_channel_is_lifestyle', ' data_channel_is_tech',
       ' data_channel_is_bus', ' data_channel_is_socmed',
       ' data_channel_is_entertainment', ' data_channel_is_world'],
      dtype=object)

In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5177 entries, 4085 to 1488
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0    n_tokens_content            5177 non-null   float64
 1    n_unique_tokens             5177 non-null   float64
 2    n_non_stop_words            5177 non-null   float64
 3    n_non_stop_unique_tokens    5177 non-null   float64
 4    num_hrefs                   5177 non-null   float64
 5    num_self_hrefs              5177 non-null   float64
 6    num_imgs                    5177 non-null   float64
 7    num_videos                  5177 non-null   float64
 8    average_token_length        5177 non-null   float64
 9    num_keywords                5177 non-null   float64
 10   global_subjectivity         5177 non-null   float64
 11   global_sentiment_polarity   5177 non-null   float64
 12   global_rate_positive_words  5177 non-null   float64
 13   global_rate_negativ